In [1]:
from neo4j import GraphDatabase

In [2]:
AURA_CONNECTION_URI = "
AURA_USERNAME = "
AURA_PASSWORD = "

driver = GraphDatabase.driver(
    AURA_CONNECTION_URI,
    auth=(AURA_USERNAME, AURA_PASSWORD)
)

In [3]:
import json

with open('food.json', 'r') as f:
    foods = json.load(f)

In [8]:
bag_of_words = {}
simple_ingredient = {}

for food, temp in foods.items():
    ingredients = temp['ingredients']
    materials = [temp.split('(')[0].strip().lower() if '(' in temp else temp.strip().lower() for temp in ingredients]

    food = food.replace('_', ' ').lower()
    simple_ingredient[food] = materials
    
    for material in materials:
        if material in bag_of_words:
            bag_of_words[material] += 1
        else:
            bag_of_words[material] = 1

print(bag_of_words)

{'apples': 1, 'sugar': 27, 'brown sugar': 2, 'cinnamon': 7, 'nutmeg': 3, 'lemon juice': 12, 'butter': 34, 'flour': 15, 'salt': 42, 'ice water': 1, 'egg wash': 1, 'pork baby back ribs': 1, 'bbq sauce': 2, 'paprika': 8, 'garlic powder': 3, 'onion powder': 1, 'salt and pepper': 16, 'cayenne pepper': 3, 'apple cider vinegar': 1, 'phyllo dough': 1, 'walnuts or pistachios': 1, 'honey': 1, 'water': 4, 'raw beef tenderloin': 2, 'olive oil': 19, 'arugula': 1, 'parmesan cheese': 9, 'capers': 2, 'egg yolk': 1, 'dijon mustard': 1, 'worcestershire sauce': 2, 'shallots': 4, 'parsley': 7, 'hot sauce': 2, 'beets': 1, 'goat cheese or feta': 1, 'arugula or mixed greens': 1, 'walnuts': 1, 'balsamic vinegar': 2, 'yeast': 1, 'milk': 9, 'eggs': 25, 'powdered sugar': 10, 'oil': 11, 'rice': 1, 'vegetables': 3, 'gochujang': 1, 'beef or tofu': 1, 'egg': 6, 'soy sauce': 12, 'sesame oil': 7, 'stale bread': 1, 'milk or cream': 2, 'vanilla extract': 18, 'raisins or nuts': 1, 'tortilla': 2, 'cheese': 8, 'potatoes': 

In [9]:
simple_ingredient

{'apple pie': ['apples',
  'sugar',
  'brown sugar',
  'cinnamon',
  'nutmeg',
  'lemon juice',
  'butter',
  'flour',
  'salt',
  'ice water',
  'egg wash'],
 'baby back ribs': ['pork baby back ribs',
  'bbq sauce',
  'brown sugar',
  'paprika',
  'garlic powder',
  'onion powder',
  'salt and pepper',
  'cayenne pepper',
  'apple cider vinegar'],
 'baklava': ['phyllo dough',
  'walnuts or pistachios',
  'sugar',
  'butter',
  'cinnamon',
  'honey',
  'water',
  'lemon juice'],
 'beef carpaccio': ['raw beef tenderloin',
  'olive oil',
  'lemon juice',
  'arugula',
  'parmesan cheese',
  'capers',
  'salt and pepper'],
 'beef tartare': ['raw beef tenderloin',
  'egg yolk',
  'capers',
  'dijon mustard',
  'worcestershire sauce',
  'shallots',
  'parsley',
  'salt and pepper',
  'hot sauce'],
 'beet salad': ['beets',
  'goat cheese or feta',
  'arugula or mixed greens',
  'walnuts',
  'balsamic vinegar',
  'olive oil',
  'salt and pepper'],
 'beignets': ['flour',
  'yeast',
  'sugar',
 

In [10]:
similar_words = {}

for words in bag_of_words.keys():
    list_words = []
    for temp in bag_of_words.keys():
        if temp.lower() != words.lower() and words.lower() in temp.lower():
            list_words.append(temp.lower())
    
    if len(list_words) > 0:
        similar_words[words.lower()] = list_words

similar_words

{'sugar': ['brown sugar', 'powdered sugar', 'granulated sugar'],
 'cinnamon': ['cinnamon sticks'],
 'lemon juice': ['lime or lemon juice'],
 'butter': ['olive oil or butter', 'butter or oil', 'buttermilk'],
 'flour': ['almond flour', 'all-purpose flour'],
 'salt': ['salt and pepper', 'sea salt'],
 'bbq sauce': ['buffalo sauce or bbq sauce'],
 'honey': ['honey or jam'],
 'water': ['ice water', 'beer or sparkling water'],
 'olive oil': ['olive oil or butter'],
 'arugula': ['arugula or mixed greens'],
 'capers': ['relish or capers'],
 'egg yolk': ['egg yolks'],
 'parsley': ['dill or parsley'],
 'walnuts': ['walnuts or pistachios'],
 'milk': ['milk or cream', 'coconut milk or heavy cream', 'buttermilk'],
 'eggs': ['hard-boiled eggs'],
 'oil': ['olive oil',
  'sesame oil',
  'vegetable oil',
  'olive oil or butter',
  'hard-boiled eggs',
  'butter or oil',
  'peanut oil'],
 'rice': ['cooked rice',
  'rice noodles',
  'arborio or short-grain rice',
  'rice vinegar',
  'arborio rice',
  'rice

In [11]:
def add_food_with_ingredients(tx, food_name, ingredients):
    query = """
    CREATE (a:Food {name: $food_name})
    WITH a
    UNWIND $ingredients AS ingredient
    MERGE (b:Ingredient {name: ingredient}) 
    MERGE (b)-[:RECIPE]->(a)
    """
    tx.run(query, food_name=food_name, ingredients=ingredients)

# Iterating through food and ingredients
for food, ingredients in simple_ingredient.items():
    with driver.session() as session:
        session.write_transaction(add_food_with_ingredients, food, ingredients)


C:\Users\Lenovo E585 (9-3)\AppData\Local\Temp\ipykernel_8232\3188449100.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_food_with_ingredients, food, ingredients)


In [12]:
def add_ingredient_relations(tx, ingredient_term, ingredient_variants):
    query = """
    MATCH (a:Ingredient {name: $ingredient_term})
    UNWIND $ingredient_variants AS variant
    MERGE (b:Ingredient {name: variant})
    MERGE (a)-[:SYNONYM]->(b)  // Consider changing to a more appropriate relationship type if necessary
    """ 
    tx.run(query, ingredient_term=ingredient_term, ingredient_variants=ingredient_variants)

for ingredient_term, ingredient_variants in similar_words.items():
    with driver.session() as session:
        session.write_transaction(add_ingredient_relations, ingredient_term, ingredient_variants)


C:\Users\Lenovo E585 (9-3)\AppData\Local\Temp\ipykernel_8232\4236040701.py:12: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_ingredient_relations, ingredient_term, ingredient_variants)
